# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [8]:
pip install azureml-train-automl-runtime==1.20.0

Note: you may need to restart the kernel to use updated packages.


In [10]:
from azureml.core import Workspace, Experiment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [11]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'auto-ml'

experiment=Experiment(ws, experiment_name)


In [13]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster-4"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [14]:
from azureml.data.dataset_factory import TabularDatasetFactory

csv_file = 'https://raw.githubusercontent.com/SwapnaKategaru/Project3/main/heart_failure_clinical_records_dataset.csv'

data = TabularDatasetFactory.from_delimited_files(csv_file)

In [16]:
%%writefile conda_dependencies.yml
dependencies:
- python=3.6.2
- pip:
  - scikit-learn=0.21.2
  - azureml-defaults

Overwriting conda_dependencies.yml


In [17]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
x, y = clean_data(data)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
train_data = pd.concat([x_train, y_train], axis = 1)

In [19]:
from azureml.train.automl import AutoMLConfig
import logging

automl_settings = {
    "enable_early_stopping" : True,
    "max_concurrent_iterations": 4,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task='classification',
    training_data=train_data,
    label_column_name='DEATH_EVENT',
    n_cross_validations=2)

In [20]:
from azureml.widgets import RunDetails
remote_run = experiment.submit(automl_config, show_output=True)
RunDetails(remote_run).show()

No run_configuration provided, running on local with default configuration

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, an

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [21]:
best_run, fitted_model = remote_run.get_output()

In [22]:
best_run.get_file_names()

['accuracy_table',
 'confusion_matrix',
 'explanation/2d61d971/classes.interpret.json',
 'explanation/2d61d971/eval_data_viz.interpret.json',
 'explanation/2d61d971/expected_values.interpret.json',
 'explanation/2d61d971/features.interpret.json',
 'explanation/2d61d971/global_names/0.interpret.json',
 'explanation/2d61d971/global_rank/0.interpret.json',
 'explanation/2d61d971/global_values/0.interpret.json',
 'explanation/2d61d971/local_importance_values.interpret.json',
 'explanation/2d61d971/per_class_names/0.interpret.json',
 'explanation/2d61d971/per_class_rank/0.interpret.json',
 'explanation/2d61d971/per_class_values/0.interpret.json',
 'explanation/2d61d971/rich_metadata.interpret.json',
 'explanation/2d61d971/visualization_dict.interpret.json',
 'explanation/eac202f6/classes.interpret.json',
 'explanation/eac202f6/eval_data_viz.interpret.json',
 'explanation/eac202f6/expected_values.interpret.json',
 'explanation/eac202f6/features.interpret.json',
 'explanation/eac202f6/global_names/0.interpret.json',
 'explanation/eac202f6/global_rank/0.interpret.json',
 'explanation/eac202f6/global_values/0.interpret.json',
 'explanation/eac202f6/local_importance_values.interpret.json',
 'explanation/eac202f6/per_class_names/0.interpret.json',
 'explanation/eac202f6/per_class_rank/0.interpret.json',
 'explanation/eac202f6/per_class_values/0.interpret.json',
 'explanation/eac202f6/rich_metadata.interpret.json',
 'explanation/eac202f6/visualization_dict.interpret.json',
 'explanation/eac202f6/ys_pred_proba_viz.interpret.json',
 'explanation/eac202f6/ys_pred_viz.interpret.json',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/internal_cross_validated_models.pkl',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

In [24]:
best_run.get_file_names()[-3]

'outputs/model.pkl'

In [25]:
import pickle

In [26]:
best_run.download_file('outputs/model.pkl', './outputs/model.pkl')

In [27]:
import joblib
joblib.load(filename='outputs/model.pkl')

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                objective=None,
                                                                                                random_state=None,
                                                                                                reg_alpha=0,
                                                        

In [28]:
from azureml.core.model import Model
model = Model.register(model_name='automl-model', model_path='outputs/model.pkl', workspace=ws)

Registering model automl-model


In [29]:
import sklearn

from azureml.core.environment import Environment

environment = Environment("LocalDeploy")
environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

In [31]:
%%writefile score.py

import json
import numpy as np
import pandas as pd
import os
import joblib
import pickle

def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
    print("Found model:", os.path.isfile(model_path)) #To check whether the model is actually present on the location we are looking at
    model = joblib.load(model_path)
def run(data):
    try:
        data = json.loads(data)['data']
        data = pd.DataFrame.from_dict(data)
        result = model.predict(data)
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

Overwriting score.py


In [32]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-139578', subscription_id='976ee174-3882-4721-b90a-b5fef6b72f24', resource_group='aml-quickstarts-139578'), name=automl-model, id=automl-model:1, version=1, tags={}, properties={})

In [33]:
Model(ws, 'automl-model')

Model(workspace=Workspace.create(name='quick-starts-ws-139578', subscription_id='976ee174-3882-4721-b90a-b5fef6b72f24', resource_group='aml-quickstarts-139578'), name=automl-model, id=automl-model:1, version=1, tags={}, properties={})

In [34]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,  enable_app_insights=True, auth_enabled=True)

from azureml.core.webservice import LocalWebservice, Webservice
inference_config = InferenceConfig(entry_script='score.py',
                                    environment=environment)
# deployment_config = LocalWebservice.deploy_configuration(port=8890)
service = Model.deploy(ws, "test1", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running......................................................................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 1111236d-cd5f-402f-890f-1f33d822b0ed
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\n\t1. Please check the logs for your container instance: test1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.\n\t2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\n\t3. You can also try to run image cb8e6bfc10894ccf9422841aba6fe7dd.azurecr.io/azureml/azureml_8a90a8ae4f21dcd7231b9244082417af locally. Please refer to https:

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 1111236d-cd5f-402f-890f-1f33d822b0ed
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\n\t1. Please check the logs for your container instance: test1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.\n\t2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\n\t3. You can also try to run image cb8e6bfc10894ccf9422841aba6fe7dd.azurecr.io/azureml/azureml_8a90a8ae4f21dcd7231b9244082417af locally. Please refe

In [ ]:
primary, secondary = service.get_keys()
print(primary)

In [ ]:
import requests
import json

scoring_uri = 'http://c05355da-4e21-43f4-999d-3893163d1a13.southcentralus.azurecontainer.io/score'

key = 'DiDq37HlNN3IuMh6wWUlSgWS4Z1KCSis'


# Sample data to send to the service
data = json.dumps(
    {"data":
            [
                {
                    'age': 75.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 582,
                    'diabetes': 0,
                    'ejection_fraction': 20,
                    'high_blood_pressure': 1,
                    'platelets': 265000.0,
                    'serum_creatinine': 1.9,
                    'serum_sodium': 130,
                    'sex': 1,
                    'smoking': 0,
                    'time': 4
                },
                {
                    'age': 55.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 7861,
                    'diabetes': 0,
                    'ejection_fraction': 38,
                    'high_blood_pressure': 0,
                    'platelets': 263358.03,
                    'serum_creatinine': 1.1,
                    'serum_sodium': 136,
                    'sex': 1,
                    'smoking': 0,
                    'time': 6
                }
            ] 
        }
)
                

headers = {'Content-Type': 'application/json'}

headers['Authorization'] = f'Bearer {key}'

resp = requests.post(scoring_uri, data, headers=headers)
print(resp.text)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [55]:
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=environment) 

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=1234)

local_service = Model.deploy(ws, "deeploy", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 88619665ccdc418b86b5c2f0b6e8e5dd.azurecr.io
Logging into Docker registry 88619665ccdc418b86b5c2f0b6e8e5dd.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 88619665ccdc418b86b5c2f0b6e8e5dd.azurecr.io/azureml/azureml_8a90a8ae4f21dcd7231b9244082417af
 ---> 0f083f3495e8
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 231fc03ba1c0
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjNkMWE1NmQyLTdjODEtNDExOC05NzkwLWY4NWQxYWNmMGM3NyIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEzOTEwNyIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEzOTEwNyIsIndvcmtzcGFjZUlkIjoiODg2MTk2NjUtY2NkYy00MThiLTg2YjUtYzJmMGI2ZThlNWRkIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 41dcb43e49ba
 ---> 61972de0099d
Step 4/5 : RUN mv '/var/azureml-app/tmpgqf2touk.py' /var/azureml-app/main.py


ERROR:azureml._model_management._util:Error: Container has crashed. Did your init method fail?



WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

In [ ]:
print('Local service port: {}'.format(local_service.port))

In [ ]:
import json

sample_input = json.dumps(
    {"data":
            [
                {
                    'age': 75.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 582,
                    'diabetes': 0,
                    'ejection_fraction': 20,
                    'high_blood_pressure': 1,
                    'platelets': 265000.0,
                    'serum_creatinine': 1.9,
                    'serum_sodium': 130,
                    'sex': 1,
                    'smoking': 0,
                    'time': 4
                },
                {
                    'age': 55.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 7861,
                    'diabetes': 0,
                    'ejection_fraction': 38,
                    'high_blood_pressure': 0,
                    'platelets': 263358.03,
                    'serum_creatinine': 1.1,
                    'serum_sodium': 136,
                    'sex': 1,
                    'smoking': 0,
                    'time': 6
                }
            ] 
        }
)

local_service.run(sample_input)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service